https://www.kaggle.com/gpiosenka/100-bird-species

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download gpiosenka/100-bird-species

Archive:  archive.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of archive.zip or
        archive.zip.zip, and cannot find archive.zip.ZIP, period.


In [ ]:
# !unzip 

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import tqdm
import os
import pandas as pd
import random
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
csv = pd.read_csv('class_dict.csv')
csv.head()

,class_index,class,height,width,scale by
0,0,AFRICAN CROWNED CRANE,224,224,1
1,1,AFRICAN FIREFINCH,224,224,1
2,2,ALBATROSS,224,224,1
3,3,ALEXANDRINE PARAKEET,224,224,1
4,4,AMERICAN AVOCET,224,224,1


In [ ]:
cls_to_int = {csv.iloc[i]['class']: csv.iloc[i]['class_index'] for i in range(len(csv))}
int_to_cls = {v:k for k, v in cls_to_int.items()}

In [ ]:
class MyDataset(Dataset):
  def __init__(self, root, train, transform):
    self.root = root
    if train:
      self.root = os.path.join(self.root, 'train')
    else:
      self.root = os.path.join(self.root, 'test')
    self.transform = transform
    self.img_path = list()
    self.labels = list()

    for bird_name in os.listdir(self.root):
      for fname in os.listdir(os.path.join(self.root, bird_name)):
        self.img_path.append(os.path.join(self.root, bird_name, fname))
        self.labels.append(cls_to_int[bird_name])

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    return self.transform(Image.open(self.img_path[idx])), self.labels[idx]

In [ ]:
train_transform = transforms.Compose([
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ColorJitter(),
                                      transforms.ToTensor()
                                      ])
test_transform = transforms.Compose([
                                     transforms.ToTensor(),
                                     ])

train_dataset = MyDataset('./', train=True, transform=train_transform)
test_dataset = MyDataset('./', train=False, transform=test_transform)

In [ ]:
BATCH_SIZE = 64

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

In [ ]:
model = models.resnet101(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(cls_to_int))
model.cuda()

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
LR = 0.0001
WD = 0.00001

optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

In [ ]:
def train(model, dataloader, optimizer):
  model.train()
  total_loss = 0
  total_acc = 0
  total_n = 0
  for x, y in tqdm.notebook.tqdm(dataloader, total=len(dataloader), desc='TRAIN', leave=False):
    optimizer.zero_grad()
    x, y = x.cuda(), y.cuda()
    logits = model(x)
    loss = F.cross_entropy(logits, y)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_acc += torch.sum(torch.argmax(logits.detach(), dim=1) == y)
    total_n += len(x)
  return total_loss / total_n, total_acc / total_n

In [ ]:
@torch.no_grad()
def test(model, dataloader):
  model.eval()
  total_loss = 0
  total_acc = 0
  total_n = 0
  for x, y, in tqdm.notebook.tqdm(dataloader, total=len(dataloader), desc='TEST', leave=False):
    x, y = x.cuda(), y.cuda()
    logits = model(x)
    loss = F.cross_entropy(logits, y)

    total_loss += loss.item()
    total_acc += torch.sum(torch.argmax(logits.detach(), dim=1) == y)
    total_n += len(x)
  return total_loss / total_n, total_acc / total_n

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
EPOCH = 50

writer = SummaryWriter('runs/exp1')
for epoch in tqdm.notebook.trange(EPOCH, desc='EPOCH'):
  train_loss, train_acc = train(model, train_dataloader, optimizer)
  test_loss, test_acc = test(model, test_dataloader)

  writer.add_scalar('train/loss', train_loss, epoch)
  writer.add_scalar('train/acc', train_acc, epoch)
  writer.add_scalar('test/loss', test_loss, epoch)
  writer.add_scalar('test/acc', test_acc, epoch)
  writer.flush()

writer.close()

EPOCH:   0%|          | 0/50 [00:00<?, ?it/s]

TRAIN:   0%|          | 0/681 [00:00<?, ?it/s]